# Práctica 5 - Regresión lineal regularizada: sesgo y varianza {-}
Álvar Domingo Fernández y Pablo Jurado López

### Preparación inicial {-}
A continuación se importan todas las librerías externas que serán utilizadas en esta práctica y se cargan los datos para hacer las distintas matrices sobre las que se va a trabajar.

In [ ]:
import numpy as np
from numpy.lib.function_base import gradient
import operator
import matplotlib.pyplot as plt
from numpy.lib.npyio import load
import scipy.optimize as opt
from scipy.io import loadmat

X = data['X']
X_orig = X
y = data['y']

X_test = data['Xtest']
X_test_orig = X_test
y_test = data['ytest']

X_val = data['Xval']
X_val_orig = X_val
y_val = data['yval']

X = np.hstack([np.ones([X.shape[0],1]),X])
X_val=np.hstack([np.ones([X_val.shape[0],1]),X_val])
X_test=np.hstack([np.ones([X_test.shape[0],1]),X_test])

thetas = np.ones(X.shape[1])

Utilizaremos durante toda la práctica las siguientes funciones para calcular el coste y el gradiente:

In [ ]:
def cost(thetas, X, Y, reg=0):
    m = X.shape[0]
    H = np.dot(X, thetas)
    cost = (1/(2*m)) * np.sum(np.square((H-Y.T))) + (reg/(2*m)) * np.sum(np.square(thetas[1:]))
    return cost

def gradient(thetas, X, Y, reg=0):
    aux = np.hstack(([0], thetas[1:]))
    m = X.shape[0]
    H = np.dot(X, thetas)
    grad = (1/m) * np.dot((H-Y.T), X) + (reg/m) * aux
    return grad

## 1- Regresión lineal regularizada {-}
Hemos utilizado la función scipy.optimize.minimize para encontrar el valor de $\theta$ que minimiza el error sobre los ejemplos de entrenamiento y posteriormente lo hemos dibujado sobre una gráfica:

In [ ]:
def apartado_1():
    print("Cost: " + str(cost(thetas, X, y, 1)))
    print("Gradient: " + str(gradient(thetas, X, y, 1)))

    reg = 0
    thetas_opt = opt.minimize(fun= cost, x0= thetas, args= (X, y, reg)).x

    plt.figure()
    plt.scatter(X[:,1], y, marker= "x", color="red")
    Y_pred = np.dot(X, thetas_opt)
    plt.plot(X[:,1], Y_pred)
    plt.xlabel('Change in water level (x)')
    plt.ylabel('Water flowing out of the dam (y)')
    plt.savefig("apartado1.png")

![Representación inicial de la recta de regresión](apartado1.png)

## 2- Curvas de aprendizaje {-}
Mediante la función get_errors se ha repetido la operación del apartado anterior utilizando diferentes subconjuntos de los ejemplos de entrenamiento (de 1 a m, siendo los subconjuntos X[:i] e y[:i]), tras lo cual se han obtenido los errores para los distintos conjuntos de entrenamiento y validación. Después se han representado gráficamente:

In [ ]:
def apartado_2():
    m = X.shape[0]
    reg = 0
    
    train_errors, val_errors = get_errors(X, y, X_val, y_val, reg)

    plt.figure()
    plt.plot(range(1, m+1), train_errors)
    plt.plot(range(1, m+1), val_errors, c='orange')
    plt.legend(("Train", "Cross Validation"))
    plt.xlabel("Number of training examples")
    plt.ylabel("Error")
    plt.savefig("apartado2.png")

![Representación de la curva de aprendizaje para la regresión lineal](apartado2.png)

Como se puede ver, el error se aproxima a los conjuntos de entrenamiento cuantos más ejemplos hay, por lo que se puede concluir que el aprendizaje está sesgado.

## 3- Regresión Polinomial {-}
